In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
import os
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.metrics import Precision, Recall
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [11]:
os.environ["WANDB_API_KEY"] = "183c1a6a36cbdf0405f5baacb72690845ecc8573"

In [12]:
train_data_dir = "dataset/mask_dataset_with_train_test/train"
val_data_dir = "dataset/mask_dataset_with_train_test/val"
test_data_dir = "dataset/mask_dataset_with_train_test/test"

train_data_gen = ImageDataGenerator( 
    rescale = 1.0/255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=(0.5, 1.5),
    validation_split=0.2
)
val_data_gen = ImageDataGenerator(
    rescale = 1.0/255.
)
test_data_gen = ImageDataGenerator(
    rescale = 1.0/255.
)

train_data_generator = train_data_gen.flow_from_directory(
    train_data_dir,
    batch_size=32,
    target_size=(224, 224)
)
val_data_generator = val_data_gen.flow_from_directory(
    val_data_dir,
    batch_size=32,
    target_size=(224, 224)
)
test_data_generator = test_data_gen.flow_from_directory(
    test_data_dir,
    batch_size=32,
    target_size=(224, 224)
)

Found 10085 images belonging to 2 classes.
Found 10086 images belonging to 2 classes.
Found 47068 images belonging to 2 classes.


In [13]:
base_model = MobileNet(include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
base_model.summary()

17235968/17225924 [==============================] - 3s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32

                                                                 
 conv_dw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_8 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9

In [14]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

In [15]:
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=['accuracy', Precision(), Recall()]
)

In [ ]:
wandb.init(project="masked_face_net", name="mobilenet_base")
wandb.config = {
    "learning_rate": 0.0001,
    "batch_size": 32,
    "epochs": 50
}
if not os.path.exists("tmp/checkpoints"):
    os.makedirs("tmp/checkpoints")

model.fit(
    train_data_generator,
    validation_data=val_data_generator,
    epochs=50,
    callbacks=[
        WandbCallback()
    ]
)

wandb: Currently logged in as: minhrongcon2000 (use `wandb login --relogin` to force relogin)


Epoch 1/50
316/316 [==============================] - 1486s 5s/step - loss: 3.1697 - accuracy: 0.8739 - precision: 0.8739 - recall: 0.8739 - val_loss: 0.1628 - val_accuracy: 0.9495 - val_precision: 0.9495 - val_recall: 0.9495 - _timestamp: 1649565604.0000 - _runtime: 1493.0000
Epoch 2/50
316/316 [==============================] - 1702s 5s/step - loss: 0.2242 - accuracy: 0.9455 - precision: 0.9455 - recall: 0.9455 - val_loss: 0.0691 - val_accuracy: 0.9799 - val_precision: 0.9799 - val_recall: 0.9799 - _timestamp: 1649567306.0000 - _runtime: 3195.0000
Epoch 3/50
316/316 [==============================] - 1817s 6s/step - loss: 0.1564 - accuracy: 0.9600 - precision: 0.9600 - recall: 0.9600 - val_loss: 0.0503 - val_accuracy: 0.9834 - val_precision: 0.9834 - val_recall: 0.9834 - _timestamp: 1649569123.0000 - _runtime: 5012.0000
Epoch 4/50
316/316 [==============================] - 1723s 5s/step - loss: 0.1208 - accuracy: 0.9641 - precision: 0.9641 - recall: 0.9641 - val_loss: 0.0740 - val_ac

In [ ]:
model.evaluate(test_data_generator)